In [1]:
#| default_exp tef_core
from nbdev import *

# TEF core framework

> contains TEF framework

In [2]:
#| export
import math
import numpy as np
import xarray as xr
import time

from pyTEF import calc

In [3]:
#| export

class constructorTEF:
    def __init__(self, inputFile, data_description, **kwargs):
        """Creates a constructor object that is passed to the pyTEF functions.

        Args:
            inputFile (str, xr.Dataset, xr.DataArray): If string is passed, function tries to read an netcdf file, otherwise xarray object can be passed as well.
            data_description (dict): Describes the data that is read

        Raises:
            IOError: Could not read filename
            IOError: No filename and invalid xarray object is passed.
            
        Example:
            data_description = {
                "lon" : None,
                "lat" : "yt_ocean",
                "depth" : "st_ocean",
                "time" : "time"
            }    
        """
        if isinstance(inputFile, str):
            try:
                self._read(inputFile, **kwargs)
            except (OSError, IOError, RunetimeError):
                raise IOError("Could not read file.")
        elif isinstance(inputFile, xr.Dataset):
            self.ds = inputFile
        elif isinstance(inputFile, xr.DataArray):
            self.ds = inputFile
        else:
            raise IOError("Please provide xarray Dataset/DataArray or netcdf-filepath")
        
        self._setup(data=data_description)
        
        self.transport = None
        self.tracer = None
        
    def _read(self, fileName, **kwargs):
        self.ds = xr.open_dataset(filename, use_cftime=True, **kwargs)
        
    def _setup(self, data):
        """Iterates over keys in dictionary. Handles 4d-data, if one argument is left empty, dummy dimension will be created.

        Args:
            data (dict): Dictionary that describes geospatial dimensions of the dataset.
        """
        for dimension in data.keys():
            if data[dimension] is None:
                self.ds = self.ds.expand_dims(dimension)
            else:
                self.ds = self.ds.rename({data[dimension]: dimension})
                                                       
        self.ds = self.ds.transpose("time",
                                    "depth",
                                    "lat",
                                    "lon")

    def sort_1dim(self, N=1024, minmaxrange=None):
        """Transform self.transport into self.tracer-coordinates with N bins in minmaxrange."""
        return calc.sort_1dim(self, N, minmaxrange)

    def sort_2dim(self, N=(1024, 1024), minmaxrange=None, minmaxrange2=None):
        """Transform self.transport into 2-dimensional self.tracer-coordinates."""
        return calc.sort_2dim(self, N, minmaxrange, minmaxrange2)

In [4]:
show_doc(constructorTEF.__init__)

<h4 id="constructorTEF.__init__" class="doc_header"><code>constructorTEF.__init__</code><a href="__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>constructorTEF.__init__</code>(**`inputFile`**, **`data_description`**, **\*\*`kwargs`**)

Creates a constructor object that is passed to the pyTEF functions.

Args:
    inputFile (str, xr.Dataset, xr.DataArray): If string is passed, function tries to read an netcdf file, otherwise xarray object can be passed as well.
    data_description (dict): Describes the data that is read

Raises:
    IOError: Could not read filename
    IOError: No filename and invalid xarray object is passed.
    
Example:
    data_description = {
        "lon" : None,
        "lat" : "yt_ocean",
        "depth" : "st_ocean",
        "time" : "time"
    }    

## Examples

In [5]:
# Try using uncommon dimensions
import xarray as xr
import numpy as np 


# Create random data
lat = np.arange(-180, 180)
lon = 37
depth = np.arange(0, 200)
time = 2
dummy_var = np.zeros((lat.size, depth.size))

ds = xr.Dataset({'dummy': (["lAtiTude", "dePht"], dummy_var)},
                coords = {'lAtiTude': (["lAtiTude"], lat),
                          'dePht': (["dePht"], depth)})

# Create data description

data_description = {
    "lon" : None,
    "lat" : "lAtiTude",
    "depth" : "dePht",
    "time" : None
}    

# Initialize TEF object

tef = constructorTEF(inputFile=ds, 
                     data_description=data_description)
tef.ds

<xarray.Dataset>
Dimensions:  (lat: 360, depth: 200, lon: 1, time: 1)
Coordinates:
  * lat      (lat) int64 -180 -179 -178 -177 -176 -175 ... 175 176 177 178 179
  * depth    (depth) int64 0 1 2 3 4 5 6 7 8 ... 192 193 194 195 196 197 198 199
Dimensions without coordinates: lon, time
Data variables:
    dummy    (time, depth, lat, lon) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0